# Тестирование Streamlit приложения

Этот ноутбук предназначен для проверки работоспособности модели и данных перед запуском Streamlit приложения.


## 1. Проверка загрузки модели


In [ ]:
import pickle
import pandas as pd
import numpy as np

# Загрузка модели
with open('model.pickle', 'rb') as f:
    model_data = pickle.load(f)

scaler = model_data['scaler']
model = model_data['model']
feature_names = model_data['feature_names']

print("✅ Модель загружена успешно!")
print(f"Тип модели: {type(model).__name__}")
print(f"Количество признаков: {len(feature_names)}")
print(f"Признаки: {feature_names}")


## 2. Проверка загрузки данных


In [ ]:
# Загрузка данных
df_train = pd.read_csv('cars_train.csv')

print(f"✅ Данные загружены успешно!")
print(f"Размер датасета: {df_train.shape}")
print(f"\nСтолбцы: {list(df_train.columns)}")
print(f"\nПервые строки:")
df_train.head()


## 3. Тестирование предсказания на примере


In [ ]:
# Создаем тестовый пример
test_data = pd.DataFrame({
    'year': [2015],
    'km_driven': [50000],
    'mileage': [20.0],
    'engine': [1200],
    'max_power': [80.0],
    'seats': [5]
})

print("Тестовые данные:")
print(test_data)

# Подготовка данных
X = test_data[feature_names]
X_scaled = scaler.transform(X)

# Предсказание
prediction = model.predict(X_scaled)[0]

print(f"\n✅ Предсказанная цена: {prediction:,.0f} ₽")


## 4. Проверка коэффициентов модели


In [ ]:
# Получаем коэффициенты
coefficients = model.coef_

coef_df = pd.DataFrame({
    'Признак': feature_names,
    'Коэффициент': coefficients
})

coef_df = coef_df.sort_values('Коэффициент', key=abs, ascending=False)

print("Коэффициенты модели:")
print(coef_df.to_string(index=False))

print(f"\nНаиболее важный признак: {coef_df.iloc[0]['Признак']}")
print(f"Его коэффициент: {coef_df.iloc[0]['Коэффициент']:,.2f}")


## 5. Проверка на нескольких примерах из датасета


In [ ]:
# Берем несколько примеров из датасета
sample_data = df_train[feature_names].head(5)

# Подготовка и предсказание
X_scaled = scaler.transform(sample_data)
predictions = model.predict(X_scaled)

# Сравнение с реальными ценами
results = pd.DataFrame({
    'Реальная цена': df_train['selling_price'].head(5).values,
    'Предсказанная цена': predictions,
    'Ошибка (%)': np.abs((predictions - df_train['selling_price'].head(5).values) / df_train['selling_price'].head(5).values * 100)
})

print("Результаты предсказания:")
print(results.to_string(index=False))
print(f"\nСредняя ошибка: {results['Ошибка (%)'].mean():.2f}%")


## 6. Инструкция по запуску Streamlit

После успешного тестирования запустите приложение:

```bash
streamlit run app.py
```

Приложение откроется в браузере по адресу `http://localhost:8501`
